In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("/kaggle/input/video-game-sales-analyze/vgsales new.csv")


In [3]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,259,Asteroids,2600,1980.0,Shooter,Atari,4.00,0.26,0.0,0.05,4.31
1,545,Missile Command,2600,1980.0,Shooter,Atari,2.56,0.17,0.0,0.03,2.76
2,1768,Kaboom!,2600,1980.0,Misc,Activision,1.07,0.07,0.0,0.01,1.15
3,1971,Defender,2600,1980.0,Misc,Atari,0.99,0.05,0.0,0.01,1.05
4,2671,Boxing,2600,1980.0,Fighting,Activision,0.72,0.04,0.0,0.01,0.77


In [4]:
df.tail()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
16593,16310,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0.00,0.0,0.0,0.01
16594,16330,Inversion,PC,NaN,Shooter,Namco Bandai Games,0.01,0.00,0.0,0.0,0.01
16595,16369,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,Unknown,0.01,0.00,0.0,0.0,0.01
16596,16430,Virtua Quest,GC,NaN,Role-Playing,Unknown,0.01,0.00,0.0,0.0,0.01
16597,16496,The Smurfs,3DS,NaN,Action,Unknown,0.00,0.01,0.0,0.0,0.01


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [6]:
df.isna().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [7]:
invalid_rows=df[df['Platform'].str.isnumeric()]
#print(invalid_rows.to_string())

In [8]:
df['Platform']=df['Platform'].replace('2600','PC')


In [9]:
#Handling null values in year

#Find the mode of the year (most common year)
most_common_year=df['Year'].mode()[0]
print(f"The year with the most game published is {most_common_year}")

df['Year'].fillna(most_common_year,inplace=True)



The year with the most game published is 2009.0


In [10]:
print(df['Year'].isna().sum())

0


In [11]:
#Handling null values in Publisher
most_common_publisher=df['Publisher'].mode()[0]
print(f"The publisher who published the most games is {most_common_publisher}")

The publisher who published the most games is Electronic Arts


In [12]:
df['Publisher'].isna().sum()

58

In [13]:
# Fill missing Publisher values with the most common Publisher for each Platform-Genre combination
df['Publisher'] = df.groupby(['Platform', 'Genre'])['Publisher'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown')
)

In [14]:
df['Publisher'].isna().sum()

0

In [15]:
print(df['Year'].head())

0    1980.0
1    1980.0
2    1980.0
3    1980.0
4    1980.0
Name: Year, dtype: float64


In [16]:
df['Year'] = df['Year'].fillna(df['Year'].mode()[0])
df['Year']=df['Year'].astype(int)

In [17]:
print(df['Year'])

0        1980
1        1980
2        1980
3        1980
4        1980
         ... 
16593    2009
16594    2009
16595    2009
16596    2009
16597    2009
Name: Year, Length: 16598, dtype: int64


In [18]:
df.isna().sum()

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64